# First try
## Prediction

In [30]:
import json
import os
from models import classMClassificator,classDetector
classifier = classMClassificator.Classificator()
detector = classDetector.Detector()
with open('sandworm.json') as fp:
    sandworm = json.load(fp=fp)

/home/d4idalos/.local/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.3.0 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[Detector]: 		  Model initializzation]		 ----------------------------------------


KeyboardInterrupt: 

In [ ]:
for entry in sandworm:
    entry['detection'] = detector.getPrediction2(entry["text"])

1/1 [==============================] - 1s 938ms/step


In [ ]:
import numpy as np
class CustomJSONizer(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.bool_):
            return bool(obj)
        elif isinstance(obj, np.float32):
            return float(obj)
        return super().default(obj)

In [ ]:
sandworm_str = json.dumps(sandworm, cls=CustomJSONizer)


In [ ]:
with open('output_detector.json', mode='w') as fp:
    json.dump(fp=fp, obj=sandworm,indent=4, cls=CustomJSONizer)

In [ ]:
for entry in sandworm:
    if entry["detection"]["response"]== True:
        entry["prediction"] = classifier.getKPrediction(entry["text"],k=3)

In [ ]:
sandworm

[{'text': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCyber-Attack Against Ukrainian Critical Infrastructure | CISA\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n \n\n\n\n\n\nAn official website of the United States government\nHere’s how you know\n\nHere’s how you know\n\n\n\n\n\n\n\n\n\nOfficial websites use .gov\n  A .gov website belongs to an official government organization in the United States.\n              \n\n\n\n\n\n\n Secure .gov websites use HTTPS\n A lock (LockA locked padlock) or https:// means you’ve safely connected to the .gov website.',
  'source': 'Cyber-Attack Against Ukrainian Critical Infrastructure _ CISA.html',
  'detection': {'response': False, 'confidence': 0.4933849},
  'prediction': {}},
 {'text': "Cybersecurity & Infrastructure Security Agency\n America's Cyber Defense Agency\n\n\n\nSearch\n\n\n\n×search\xa0\n\n\nMenu\n\n\n\n\n\n\nClose\n\n\n×search\xa0\n\n\n\n\nTopics\n\n\n\nTopics\n\n\n\n\n\nCybersecurity Best 

In [ ]:
with open('output/sandworm.json', mode='w') as fp:
    json.dump(fp=fp, obj=sandworm,indent=4, cls=CustomJSONizer)

## Evaluation
### Retrieve from Caldera and clean

In [ ]:
def remove_subteq(lst:list)->list:
    result = [code.split('.')[0] for code in lst]
    return result

In [ ]:
import pandas as pd
adv_profile = pd.read_csv('data/caldera/adversary_profiles.csv')
adv_profile.head(5)
adv_sandworm = eval(adv_profile.loc[adv_profile['Adversary name'] == 'Sandworm Team (G0034)', 'Technique list'].values[0])
adv_sandworm = remove_subteq(adv_sandworm)


In [33]:
from sorter import classSorter


['T1105',
 'T1505',
 'T1033',
 'T1082',
 'T1083',
 'T1003',
 'T1105',
 'T1548',
 'T1548',
 'T1548',
 'T1053',
 'T1543',
 'T1003',
 'T1552',
 'T1552',
 'T1021',
 'T1569',
 'T1547',
 'T1547',
 'T1033',
 'T1082',
 'T1083',
 'T1049',
 'T1105',
 'T1555',
 'T1105',
 'T1056',
 'T1056',
 'T1056',
 'T1041',
 'T1056',
 'T1018',
 'T1070',
 'T1021',
 'T1105',
 'T1021',
 'T1105',
 'T1105',
 'T1486',
 'T1021']

In [ ]:
sorter_c_sandworm = classSorter.Sorter(tactics_looktable_path='sorter/data/lookup_MITRE_Enterprise.csv', index_looktable_path='sorter/data/sort_table.csv' , techniques= adv_sandworm)

## Retrieve tech list

In [ ]:
sandworm_list =[]                                                       # top k techniques predicted
for entry in sandworm:
    if entry["detection"]["response"] == True:
        sandworm_list.append(entry["prediction"]["attack_id"])


In [ ]:
sandworm_tops = []                                                      # predicted list
for techniques in sandworm_list:
    sandworm_tops.append(techniques[0])

### Old criterion


In [ ]:
sorter_sandworm = classSorter.Sorter(tactics_looktable_path='sorter/data/lookup_MITRE_Enterprise.csv', index_looktable_path='sorter/data/sort_table.csv' , techniques= sandworm_tops)

In [ ]:
def true_positive_p(list1: list, list2: list)->float:
    A = remove_subteq(list1)
    B = remove_subteq(list2)
    As = set(A)
    Bs = set(B)
    union = As | Bs
    inter = As & Bs
    return len(inter)/len(Bs)

def false_negative_p(list1: list, list2: list)->float:
    A = remove_subteq(list1)
    B = remove_subteq(list2)
    As = set(A)
    Bs = set(B)
    union = As | Bs
    inter = As & Bs
    return len(Bs-As)/len(Bs)

def false_positive_p(list1: list, list2: list)->float:
    A = remove_subteq(list1)
    B = remove_subteq(list2)
    As = set(A)
    Bs = set(B)
    union = As | Bs
    inter = As & Bs
    return len(As-Bs)/len(As)


def debug_false_negative_p(list1: list, list2: list)->float:
    A = remove_subteq(list1)
    B = remove_subteq(list2)
    As = set(A)
    Bs = set(B)
    union = As | Bs
    inter = As & Bs
    print(As)
    print(Bs)
    print(As^Bs)
    return len(Bs-As)/len(Bs)

In [32]:
print("Confronto tra tattiche")
print("la percentuale di fit per Sandworm è:\t", true_positive_p(sorter_sandworm.get_sorted_tactics(), sorter_c_sandworm.get_sorted_tactics()))
print("la percentuale di fit per Sandworm è:\t",  false_positive_p(sorter_sandworm.get_sorted_tactics(), sorter_c_sandworm.get_sorted_tactics()))

Confronto tra tattiche
la percentuale di fit per Sandworm è:	 1.0
la percentuale di fit per Sandworm è:	 0.21428571428571427


In [31]:
print("confronto tra tecniche")
print("la percentuale di fit per Sandworm è:\t", true_positive_p(sorter_sandworm.get_sorted_techniques(), sorter_c_sandworm.get_sorted_techniques()))
print("la percentuale di fit per Sandworm è:\t", false_positive_p(sorter_sandworm.get_sorted_techniques(), sorter_c_sandworm.get_sorted_techniques()))

confronto tra tecniche
la percentuale di fit per Sandworm è:	 0.6
la percentuale di fit per Sandworm è:	 0.8208955223880597


## Nuovo criterio


In [36]:
cleaned_adv_sandworm= list(dict.fromkeys(adv_sandworm))

In [99]:
def tuple_true_positives(topk_list:list, positive_list:list)->list:
    # reputa un true positive se è presente almeno un elemento della tupla nella lista caldera e restituisce il primo elemento che ha dato esito positivo (più probabile)
    true_positives = []
    for klist in topk_list:
        for techniques in klist:
            for adv in positive_list:
                if(techniques)==adv:
                    true_positives.append(techniques)
    return true_positives
                

In [104]:
def tuple_false_positives(topk_list:list, positive_list:list)->list:
    # reputa un false positive quella tupla che non ha alcuna corrispondenza (restituisce pertanto una lista composta da tuple k-dimensionalis)
    false_positives = []
    for klist in topk_list:
        acc = 0
        for techniques in klist:
            for adv in positive_list:
                if(techniques)==adv:
                    acc = acc +1
        if acc == 0:
            false_positives.append(klist)
    return false_positives

def len_tuple(klist:list)-> int:
    #rimuove le tuple duplicate e restituisce la dimensione di insieme
    return len(list(map(list, set(map(tuple, klist)))))



In [107]:
tp= tuple_true_positives(topk_list=sandworm_list,positive_list=cleaned_adv_sandworm)
tp = list(dict.fromkeys(tp))
fp= tuple_false_positives(topk_list=sandworm_list,positive_list=cleaned_adv_sandworm)

print("Sandworm scores:")
print("True positives on the actual positives:\t", len(tp)/len(cleaned_adv_sandworm))
print("False positives (tuple) on predicted:\t", len_tuple(fp)/len(sandworm_list))


Sandworm scores:
True positives on the actual positives:	 0.8
False positives (tuple) on predicted:	 0.3776824034334764


# Ricomposizione esempio con il solo tokenizer Spacy

In [108]:
import json
import os
from models import classMClassificator,classDetector
classifier = classMClassificator.Classificator()
detector = classDetector.Detector()
with open('sandworm_naive.json') as fp:
    sandworm = json.load(fp=fp)

/home/d4idalos/.local/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.3.0 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[Detector]: 		  Model initializzation]		 ----------------------------------------


/home/d4idalos/.local/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [109]:
for entry in sandworm:
    entry['detection'] = detector.getPrediction2(entry["text"])

1/1 [==============================] - 1s 659ms/step


In [110]:
for entry in sandworm:
    if entry["detection"]["response"]== True:
        entry["prediction"] = classifier.getKPrediction(entry["text"],k=3)

In [111]:
with open('output/sandworm_naive.json', mode='w') as fp:
    json.dump(fp=fp, obj=sandworm,indent=4, cls=CustomJSONizer)

In [112]:
sandworm_list =[]                                                       # top k techniques predicted
for entry in sandworm:
    if entry["detection"]["response"] == True:
        sandworm_list.append(entry["prediction"]["attack_id"])

In [118]:
tp= tuple_true_positives(topk_list=sandworm_list,positive_list=cleaned_adv_sandworm)
tp = list(dict.fromkeys(tp))
fp= tuple_false_positives(topk_list=sandworm_list,positive_list=cleaned_adv_sandworm)

print("Sandworm scores:")
print("True positives on the actual positives:\t", len(tp)/len(cleaned_adv_sandworm))
print("False positives (tuple) on predicted:\t", len(fp)/len(sandworm_list))

Sandworm scores:
True positives on the actual positives:	 0.95
False positives (tuple) on predicted:	 0.4121951219512195


In [119]:
print(len(fp))
len_tuple(fp)

338


325

In [120]:
len(sandworm_list)

820

# Esperimenti senza rebuild

In [121]:
import json
with open('output/sandworm.json') as fp:
    sandworm = json.load(fp=fp)
with open('output/sandworm_naive.json') as fp:
    sandworm_naive = json.load(fp=fp)

In [123]:
def tuple_false_positives(topk_list:list, positive_list:list)->list:
    # reputa un false positive quella tupla che non ha alcuna corrispondenza (restituisce pertanto una lista composta da tuple k-dimensionalis)
    false_positives = []
    for klist in topk_list:
        acc = 0
        for techniques in klist:
            for adv in positive_list:
                if(techniques)==adv:
                    acc = acc +1
        if acc == 0:
            false_positives.append(klist)
    return false_positives

def tuple_true_positives(topk_list:list, positive_list:list)->list:
    # reputa un true positive se è presente almeno un elemento della tupla nella lista caldera e restituisce il primo elemento che ha dato esito positivo (più probabile)
    true_positives = []
    for klist in topk_list:
        for techniques in klist:
            for adv in positive_list:
                if(techniques)==adv:
                    true_positives.append(techniques)
    return true_positives

In [127]:
sandworm_list =[]                                                       # top k techniques predicted
for entry in sandworm:
    if entry["detection"]["response"] == True:
        sandworm_list.append(entry["prediction"]["attack_id"])

In [126]:
sandworm_list_naive =[]                                                       # top k techniques predicted
for entry in sandworm_naive:
    if entry["detection"]["response"] == True:
        sandworm_list_naive.append(entry["prediction"]["attack_id"])

In [128]:
tp= tuple_true_positives(topk_list=sandworm_list,positive_list=cleaned_adv_sandworm)
tp = list(dict.fromkeys(tp))
fp= tuple_false_positives(topk_list=sandworm_list,positive_list=cleaned_adv_sandworm)

print("Sandworm scores:")
print("True positives on the actual positives:\t", len(tp)/len(cleaned_adv_sandworm))
print("False positives (tuple) on predicted:\t", len(fp)/len(sandworm_list))

Sandworm scores:
True positives on the actual positives:	 0.8
False positives (tuple) on predicted:	 0.38197424892703863


In [131]:
tp_naive= tuple_true_positives(topk_list=sandworm_list_naive,positive_list=cleaned_adv_sandworm)
tp_naive = list(dict.fromkeys(tp_naive))
fp_naive= tuple_false_positives(topk_list=sandworm_list_naive,positive_list=cleaned_adv_sandworm)

print("Sandworm naive chunking scores:")
print("True positives on the actual positives:\t", len(tp_naive)/len(cleaned_adv_sandworm))
print("False positives (tuple) on predicted:\t", len(fp_naive)/len(sandworm_list_naive))

Sandworm naive chunking scores:
True positives on the actual positives:	 0.95
False positives (tuple) on predicted:	 0.4121951219512195
